In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras

### Importing Dataset From Directory

In [3]:
train_dir=r'D:\Internshala Videos\Deep Learning\Internshala ipynb\IR Project Data\Data\cats_and_dogs\train'
test_dir=r'D:\Internshala Videos\Deep Learning\Internshala ipynb\IR Project Data\Data\cats_and_dogs\test'
validation_dir=r'D:\Internshala Videos\Deep Learning\Internshala ipynb\IR Project Data\Data\cats_and_dogs\validation'

## VGG16 Model

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


#### Convolutional Base

In [5]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [6]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [7]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

c:\Users\Rohit\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [10]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("CNN_Project_Model-{epoch:02d}.h5")

In [11]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=[checkpoint_cb])

C:\Users\Rohit\AppData\Local\Temp\ipykernel_57032\2272497544.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
100/100 [==============================] - 577s 6s/step - loss: 0.5264 - acc: 0.7295 - val_loss: 0.2383 - val_acc: 0.9230
Epoch 2/30
100/100 [==============================] - 587s 6s/step - loss: 0.2853 - acc: 0.8760 - val_loss: 0.2051 - val_acc: 0.9250
Epoch 3/30
100/100 [==============================] - 582s 6s/step - loss: 0.2388 - acc: 0.8920 - val_loss: 0.1684 - val_acc: 0.9320
Epoch 4/30
100/100 [==============================] - 646s 6s/step - loss: 0.2049 - acc: 0.9145 - val_loss: 0.2099 - val_acc: 0.9290
Epoch 5/30
100/100 [==============================] - 875s 9s/step - loss: 0.1408 - acc: 0.9460 - val_loss: 0.1038 - val_acc: 0.9590
Epoch 6/30
100/100 [==============================] - 627s 6s/step - loss: 0.1612 - acc: 0.9365 - val_loss: 0.1117 - val_acc: 0.9600
Epoch 7/30
 74/100 [=====================>........] - ETA: 3:21 - loss: 0.1132 - acc: 0.9527

: 

: 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
hist_df = pd.DataFrame(history.history) 

In [ ]:
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

In [ ]:
model.evaluate_generator(test_generator, steps=50)